<a href="https://colab.research.google.com/github/eduard-ignatev/high-performance-python/blob/main/04_hw/metro_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://raw.githubusercontent.com/ninadicara/tube-journeys/refs/heads/master/london.stations.csv
!wget https://raw.githubusercontent.com/ninadicara/tube-journeys/refs/heads/master/london.connections.csv

--2025-03-24 19:15:42--  https://raw.githubusercontent.com/ninadicara/tube-journeys/refs/heads/master/london.stations.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17233 (17K) [text/plain]
Saving to: ‘london.stations.csv.2’

london.stations.csv 100%[===================>]  16.83K  --.-KB/s    in 0s      

2025-03-24 19:15:42 (68.5 MB/s) - ‘london.stations.csv.2’ saved [17233/17233]

--2025-03-24 19:15:42--  https://raw.githubusercontent.com/ninadicara/tube-journeys/refs/heads/master/london.connections.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting

In [2]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse

In [3]:
stations_df = pd.read_csv('london.stations.csv')
connections_df = pd.read_csv('london.connections.csv')

stations = sorted(set(connections_df['station1'].unique()) | set(connections_df['station2'].unique()))
station_id_to_index = {station: i for i, station in enumerate(stations)}
station_index_to_id = {i: station for i, station in enumerate(stations)}
station_id_to_name = dict(zip(stations_df['id'], stations_df['name']))

# Create empty adjacency matrix
n_stations = len(stations)
adj_matrix = np.zeros((n_stations, n_stations), dtype=int)

# Fill in connections
for _, row in connections_df.iterrows():
    station1_idx = station_id_to_index[row['station1']]
    station2_idx = station_id_to_index[row['station2']]

    # Set connection in both directions
    adj_matrix[station1_idx, station2_idx] = 1
    adj_matrix[station2_idx, station1_idx] = 1

adj_matrix = sparse.csr_matrix(adj_matrix)

In [4]:
adj_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 698 stored elements and shape (302, 302)>

In [5]:
# Print station info
station_id = 11
station_name = station_id_to_name[station_id]
print(f"Station ID: {station_id}, Station Name: {station_name}")

station_idx = station_id_to_index[station_id]
connected_indices = adj_matrix[station_idx].nonzero()[1]
connected_stations = [station_id_to_name[station_index_to_id[idx]] for idx in connected_indices]
print(f"Connected stations: {connected_stations}")

Station ID: 11, Station Name: Baker Street
Connected stations: ['Bond Street', 'Edgware Road (C)', 'Finchley Road', 'Great Portland Street', 'Marylebone', "Regent's Park", "St. John's Wood"]


In [6]:
# Print station info
station_id = 25
station_name = station_id_to_name[station_id]
print(f"Station ID: {station_id}, Station Name: {station_name}")

station_idx = station_id_to_index[station_id]
connected_indices = adj_matrix[station_idx].nonzero()[1]
connected_stations = [station_id_to_name[station_index_to_id[idx]] for idx in connected_indices]
print(f"Connected stations: {connected_stations}")

Station ID: 25, Station Name: Blackfriars
Connected stations: ['Mansion House', 'Temple']


## PageRank

In [7]:
def pagerank_iterative(adjacency_matrix, alpha=0.85, max_iterations=100, tol=1e-6):
    """
    Computes the PageRank vector for a given adjacency matrix.

    Parameters:
    -----------
    adjacency_matrix : numpy.ndarray
        The adjacency matrix where adjacency_matrix[i,j] = 1 if there is a link from page i to page j
    alpha : float
        The teleportation probability (damping factor), usually set to 0.85
    max_iterations : int
        Maximum number of iterations for the power method
    tol : float
        Convergence tolerance for the power method

    Returns:
    --------
    numpy.ndarray
        The PageRank vector where element i is the PageRank score of page i
    int
        Number of iterations performed
    """
    n = adjacency_matrix.shape[0]

    # Create the column-stochastic transition matrix
    outbound_links = adjacency_matrix.sum(axis=1)
    transition_matrix = np.zeros((n, n))

    for i in range(n):
        if outbound_links[i] > 0:
            transition_matrix[i,:] = adjacency_matrix[i,:] / outbound_links[i]
        else:
            # If a page has no outgoing links, distribute evenly to all pages
            transition_matrix[i,:] = np.ones(n) / n

    # Transpose to get the correct Markov transition matrix
    transition_matrix = transition_matrix.T

    # Initialize the PageRank vector
    pagerank_vector = np.ones(n) / n

    # Apply the power method
    for iteration in range(max_iterations):
        # Store the previous PageRank vector to check for convergence
        previous_pagerank = pagerank_vector.copy()

        # Compute the new PageRank vector
        # Formula: PR = alpha * (transition_matrix * PR) + (1-alpha) * (1/n)
        pagerank_vector = alpha * np.dot(transition_matrix, pagerank_vector) + (1 - alpha) * np.ones(n) / n

        # Check for convergence
        if np.linalg.norm(pagerank_vector - previous_pagerank, 1) < tol:
            return pagerank_vector, iteration + 1

    return pagerank_vector, max_iterations

In [8]:
alpha = 0.85

pr_vector, iterations = pagerank_iterative(adj_matrix.toarray(), alpha)

print(f"PageRank converged after {iterations} iterations.")
print(f"Sum of PageRank scores: {pr_vector.sum():.6f}")

PageRank converged after 65 iterations.
Sum of PageRank scores: 1.000000


In [9]:
df_pagerank = pd.DataFrame(pr_vector, columns=['pagerank_score'])
df_pagerank['station'] = df_pagerank.index.to_series().map(station_index_to_id).map(station_id_to_name)
df_pagerank.sort_values(by='pagerank_score', ascending=False).head(5)

,pagerank_score,station
144,0.007921,King's Cross St. Pancras
10,0.007614,Baker Street
73,0.007047,Earl's Court
12,0.006196,Bank
191,0.006178,Paddington


## Sparse vs Dense



In [10]:
def pagerank_algebraic(A, p=0.85, personalize=None, reverse=False):
    """ Calculates PageRank given a csr graph

    Inputs:
    -------

    G: a csr graph.
    p: damping factor
    personlize: if not None, should be an array with the size of the nodes
                containing probability distributions.
                It will be normalized automatically
    reverse: If true, returns the reversed-PageRank

    outputs
    -------

    PageRank Scores for the nodes

    """
    # In Moler's algorithm, $$A_{ij}$$ represents the existences of an edge
    # from node $$j$$ to $$i$$, while we have assumed the opposite!
    if reverse:
        A = A.T

    n, _ = A.shape
    r = np.asarray(A.sum(axis=1)).reshape(-1)

    k = r.nonzero()[0]

    D_1 = sparse.csr_matrix((1 / r[k], (k, k)), shape=(n, n))

    if personalize is None:
        personalize = np.ones(n)
    personalize = personalize.reshape(n, 1)
    s = (personalize / personalize.sum()) * n

    I = sparse.eye(n)
    x = sparse.linalg.spsolve((I - p * A.T @ D_1), s)

    x = x / x.sum()
    return x

In [11]:
def generate_sparse_adjacency_matrix(num_nodes, density):
  random_matrix = np.random.rand(num_nodes, num_nodes)
  adjacency_matrix = (random_matrix < density).astype(int)
  adjacency_matrix = np.maximum(adjacency_matrix, adjacency_matrix.transpose())
  sparse_adjacency_matrix = sparse.csr_matrix(adjacency_matrix)
  return sparse_adjacency_matrix

In [12]:
num_nodes = 1000
density = 0.001
sparse_adj_matrix = generate_sparse_adjacency_matrix(num_nodes, density)
sparse_adj_matrix

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 1952 stored elements and shape (1000, 1000)>

In [13]:
%%timeit
pagerank_algebraic(sparse_adj_matrix)

5.89 ms ± 520 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [15]:
%%timeit
pagerank_iterative(sparse_adj_matrix.toarray())

48.5 ms ± 1.42 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
